### The Robot - ID correspondence
0 -> R1
1 -> R2
2 -> B1
3 -> B2

### size of the scene

808 (x) * 450 (z)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
#
# %cd "drive/MyDrive"
# %ls
#
# !nvidia-smi

# !unzip "drive/MyDrive/data.zip" -d "drive/MyDrive"

In [69]:
# import useful packages
import math
import statistics
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.nn.functional as f
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time

# set pandas display option
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

# set device
device = torch.device('cuda')

In [60]:
def read_inputs():

    inputs = pd.DataFrame()

    # read input yolo data
    for label_file in Path("data/inputs").iterdir():
        # read labels
        df = pd.read_csv(label_file,sep=" ",names=['id','center-x','center-y','w','h'])

        image_type = str(label_file.name).split('_')

        df['picture'] = [int(image_type[0])] * len(df.index)

        if image_type[1] == "Screen1":
            df['screen'] = [1] * len(df.index)
        elif image_type[1] == "Screen2":
            df['screen'] = [2] * len(df.index)

        inputs = inputs.append(df, ignore_index=True)

    inputs = inputs.sort_values(by=["picture","id", "screen"]).reset_index(drop=True)

    return inputs

inputs = read_inputs()
print(len(inputs.index))
display(inputs)

3495


,id,center-x,center-y,w,h,picture,screen
0,0,0.372656,0.388194,0.060937,0.087500,1,1
1,0,0.338281,0.737500,0.145313,0.225000,1,2
2,1,0.151953,0.485417,0.064844,0.115278,1,1
3,1,0.589453,0.482639,0.089844,0.129167,1,2
4,2,0.585938,0.456250,0.078125,0.120833,1,1
5,2,0.108594,0.520139,0.092188,0.131944,1,2
6,3,0.399609,0.797222,0.153906,0.244444,1,1
7,3,0.350000,0.363889,0.059375,0.077778,1,2
8,0,0.376563,0.389583,0.064062,0.084722,2,1
9,0,0.327344,0.722917,0.140625,0.215278,2,2


In [61]:
'''
process input data

data format is [x1, y1, h1, x2, y2, h2]
'''
def process_input():

    input_data = []

    for i in range(1,501):
        for j in range(0,4):

            temp = inputs.loc[(inputs['picture'] == i) & (inputs['id'] == j)].reset_index(drop=True)

            if len(temp.index) == 0:
                temp1 = [0]
                # temp1 = [0,0,0,0,0,0]
            elif len(temp.index) == 1:
                if temp.iloc[0]['screen'] == 1:
                    temp1 = [0]
                    # temp1 = [float(temp['center-x']), float(temp['center-y']), float(temp['h']), 0, 0, 0]
                else:
                    temp1 = [0]
                    # temp1 = [0, 0, 0, float(temp['center-x']), float(temp['center-y']), float(temp['h'])]
            else:
                assert (len(temp.index) == 2)
                temp1 = []
                for index, row in temp.iterrows():
                    temp1 += [float(row['center-x']), float(row['center-y']), float(row['h'])]

            input_data.append(temp1)

    return input_data

input_data = process_input()
print(len(input_data))
print(input_data[0:20])

2000
[[0.37265625, 0.3881944444444444, 0.0875, 0.33828125, 0.7375, 0.225], [0.151953125, 0.4854166666666666, 0.1152777777777777, 0.589453125, 0.4826388888888889, 0.1291666666666666], [0.5859375, 0.45625, 0.1208333333333333, 0.10859375, 0.5201388888888889, 0.1319444444444444], [0.399609375, 0.7972222222222223, 0.2444444444444444, 0.35, 0.3638888888888889, 0.0777777777777777], [0.3765625, 0.3895833333333333, 0.0847222222222222, 0.32734375, 0.7229166666666667, 0.2152777777777778], [0.153515625, 0.4895833333333333, 0.1180555555555555, 0.582421875, 0.4756944444444445, 0.1291666666666666], [0.5964843750000001, 0.4618055555555556, 0.1208333333333333, 0.10390625, 0.5131944444444445, 0.1291666666666666], [0.38125, 0.8152777777777778, 0.2527777777777777, 0.357421875, 0.3590277777777778, 0.0847222222222222], [0.38046875, 0.3895833333333333, 0.0930555555555555, 0.320703125, 0.7069444444444445, 0.2166666666666666], [0.156640625, 0.4958333333333333, 0.1194444444444444, 0.571875, 0.46875, 0.129166666

In [62]:
def read_outputs():

    df1 = pd.read_csv("./data/labels/test_gt_coordinates.csv")
    df2 = pd.read_csv("./data/labels/train_gt_coordinates.csv")
    df3 = pd.read_csv("./data/labels/val_gt_coordinates.csv")

    outputs = df1.append(df2, ignore_index=True).append(df3, ignore_index=True).sort_values(by="ID")
    outputs.columns = ["picture", "2.x", "2.z", "3.x", "3.z", "0.x", "0.z", "1.x", "1.z"]

    outputs_0 = outputs.loc[:,['picture','0.x','0.z']]
    outputs_0['id'] = [0] * 500
    outputs_0.columns = ['picture','x', 'z', 'id']

    outputs_1 = outputs.loc[:,['picture','1.x','1.z']]
    outputs_1['id'] = [1] * 500
    outputs_1.columns = ['picture','x', 'z', 'id']

    outputs_2 = outputs.loc[:,['picture','2.x','2.z']]
    outputs_2['id'] = [2] * 500
    outputs_2.columns = ['picture','x', 'z', 'id']

    outputs_3 = outputs.loc[:,['picture','3.x','3.z']]
    outputs_3['id'] = [3] * 500
    outputs_3.columns = ['picture','x', 'z', 'id']

    outputs = outputs_2.append(outputs_3, ignore_index=True).append(outputs_0, ignore_index=True).append(outputs_1, ignore_index=True).sort_values(by = ['picture','id']).reset_index(drop=True)

    return outputs

outputs = read_outputs()
print(len(outputs))
display(outputs)

2000


,picture,x,z,id
0,1,289.79580,56.15300,0
1,1,189.15910,334.19810,1
2,1,578.79880,99.15597,2
3,1,555.80790,341.15390,3
4,2,279.62180,56.68040,0
5,2,189.69620,344.36450,1
6,2,568.62970,99.68863,2
7,2,545.65060,341.68140,3
8,3,269.52640,57.55946,0
9,3,190.58840,354.44900,1


In [63]:
'''
Convert Data Format of output data
data format is: [x, z]
'''

def process_outputs():
    output_data = []

    for index, row in outputs.iterrows():
        output_data.append([row['x']/808, row['z']/450])

    return output_data

output_data = process_outputs()
print(len(output_data))
print(output_data[0:20])

2000
[[0.35865816831683167, 0.12478444444444445], [0.23410779702970297, 0.7426624444444445], [0.7163351485148515, 0.2203466], [0.6878810643564357, 0.7581197777777778], [0.34606658415841585, 0.12595644444444445], [0.23477252475247526, 0.7652544444444445], [0.7037496287128713, 0.2215302888888889], [0.6753101485148516, 0.759292], [0.3335722772277228, 0.1279099111111111], [0.23587673267326734, 0.7876644444444445], [0.6912644801980198, 0.2235008888888889], [0.6628419554455445, 0.7612417777777778], [0.32115, 0.1306392], [0.23741670792079206, 0.8099393333333332], [0.678854702970297, 0.226252], [0.6504512376237624, 0.7639651111111111], [0.30878564356435645, 0.1341494888888889], [0.23939393564356434, 0.8321082222222221], [0.6665032178217821, 0.22978644444444443], [0.638121905940594, 0.7674631111111111]]


In [64]:
'''
Train test split and convert to tensor
'''
input_data_1 = []
output_data_1 = []
for i in range(len(input_data)):
    if input_data[i] != [0]:
        input_data_1.append(input_data[i])
        output_data_1.append(output_data[i])

print(input_data_1)
# output_data = output_data_1

# x_train, x_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.2, random_state=42)
input_data = torch.tensor(input_data_1, dtype=torch.float32).to(device)
output_data = torch.tensor(output_data_1, dtype=torch.float32).to(device)

torch.manual_seed(seed=1)

torch_dataset = TensorDataset(input_data, output_data)
train_data, val_test = random_split(torch_dataset,
                                     [1207, 302])
val_data, test_data = random_split(val_test,
                                     [151, 151])

train_data = DataLoader(train_data,
                        batch_size=6,
                        shuffle=True)

[[0.37265625, 0.3881944444444444, 0.0875, 0.33828125, 0.7375, 0.225], [0.151953125, 0.4854166666666666, 0.1152777777777777, 0.589453125, 0.4826388888888889, 0.1291666666666666], [0.5859375, 0.45625, 0.1208333333333333, 0.10859375, 0.5201388888888889, 0.1319444444444444], [0.399609375, 0.7972222222222223, 0.2444444444444444, 0.35, 0.3638888888888889, 0.0777777777777777], [0.3765625, 0.3895833333333333, 0.0847222222222222, 0.32734375, 0.7229166666666667, 0.2152777777777778], [0.153515625, 0.4895833333333333, 0.1180555555555555, 0.582421875, 0.4756944444444445, 0.1291666666666666], [0.5964843750000001, 0.4618055555555556, 0.1208333333333333, 0.10390625, 0.5131944444444445, 0.1291666666666666], [0.38125, 0.8152777777777778, 0.2527777777777777, 0.357421875, 0.3590277777777778, 0.0847222222222222], [0.38046875, 0.3895833333333333, 0.0930555555555555, 0.320703125, 0.7069444444444445, 0.2166666666666666], [0.156640625, 0.4958333333333333, 0.1194444444444444, 0.571875, 0.46875, 0.12916666666666

In [65]:
'''
define the regression model to regress outcomes
'''
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.input = nn.Linear(6, 10)
        self.h1 = nn.Linear(10,100)
        self.h2 = nn.Linear(100,50)
        self.output = nn.Linear(50, 2)

    def forward(self, x):
        out = self.input(x)
        out  = torch.relu(out)
        out = self.h1(out)
        out = torch.relu(out)
        out = self.h2(out)
        out = torch.relu(out)
        out = self.output(out)
        return out

In [70]:
'''
train the model
'''
lr = 1e-5
epochs = 50
start_time = time.time()

model = Model()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr)  # 随机梯度下降法
criterion = nn.MSELoss()  # 均方差损失函数

for epoch in range(epochs):
    model.train()
    loses = []

    for _, (train_input, train_output) in enumerate(train_data):

        out = model.forward(train_input) # 计算模型输出结果
        loss = criterion(out, train_output)  # 损失函数
        loses.append(loss.item())
        optimizer.zero_grad()  # 梯度清零
        loss.backward()  # 计算梯度 后向传递过程
        optimizer.step() # 优化权重与偏差矩阵

    logit = []  # 这个是验证集，可以根据验证集的结果进行调参，这里根据验证集的结果选取最优的神经网络层数与神经元数目
    target = []
    model.eval() # 启动测试模式
    for data, targets in val_data:  # 输出验证集的平均误差
        logits = model.forward(data).detach().cpu().numpy()
        targets=targets.detach().cpu().numpy()
        logit.append(logits[0])
        target.append(targets[0])
    average_loss =  criterion(torch.tensor(logit), torch.tensor(target))

    if epoch%100 == 0:
      print(epoch, average_loss, sum(loses)/len(loses), sep='\t')

end_time = time.time()
print("Time usage is %d", (end_time - start_time))

0	tensor(0.2401)	0.2619113534283225
Time usage is %d -13.168723583221436


In [ ]:
'''
test the model
'''
predictions = []
targets = []
for _, (test_input, test_output) in enumerate(test_data):

    predictions.append(model.forward(test_input).detach().tolist())
    targets.append(test_output.detach().tolist())

print(len(predictions))
print(len(targets))

# measure the outcome
outcome = []

for i in range(0,200):
    [x1, z1] = predictions[i]
    [x2, z2] = targets[i]
    # outcome.append(abs((x1-x2)*808))
    outcome.append(math.sqrt(((x1-x2)*808)**2 + ((z1-z2)*450)**2))

print("the mean deviation is:")
print(statistics.mean(outcome))